In [6]:
!pip install pymupdf pillow

   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ------------------ --------------------- 8.7/18.7 MB 48.3 MB/s eta 0:00:01
   ---------------------------------------  18.6/18.7 MB 45.0 MB/s eta 0:00:01
   ---------------------------------------- 18.7/18.7 MB 40.1 MB/s eta 0:00:00


In [15]:
import os
from pathlib import Path
import fitz  # PyMuPDF
from PIL import Image

#  Welcome message
print("✨ Welcome to the PDF to Resized Images Converter! ✨")
print("Let's turn your PDFs into resized images.\n")

# ---------------------------
# 📁 Ask user for paths
# ---------------------------
input_folder = input("📂 Enter the path to the folder containing PDFs: ").strip()
base_output = input("📂 Enter the path where resized images should be saved: ").strip()

# ---------------------------
# 🎨 Ask user for target sizes
# ---------------------------
sizes = {}
print("\n🎨 Now, let's define your target image sizes.")
while True:
    size_input = input("➡️ Enter size (WIDTHxHEIGHT) or 0 to finish: ").strip()
    if size_input == "0":
        break
    try:
        width, height = map(int, size_input.lower().split('x'))
        folder_name = f"{width}x{height}"
        sizes[(width, height)] = folder_name
        print(f"✅ Added size: {folder_name}\n")
    except ValueError:
        print("⚠️ Please enter a valid size in the format WIDTHxHEIGHT (e.g., 200x200).\n")

if not sizes:
    print("❌ No sizes defined. Exiting.")
    exit()

# ---------------------------
# 🛠️ Make sure all output folders exist
# ---------------------------
for folder in sizes.values():
    os.makedirs(os.path.join(base_output, folder), exist_ok=True)

# ---------------------------
# 🔧 Crop & resize function
# ---------------------------
def crop_center_resize(img, target_size):
    """Crop image to keep aspect ratio then resize to target size."""
    target_w, target_h = target_size
    img_ratio = img.width / img.height
    target_ratio = target_w / target_h

    if img_ratio > target_ratio:  
        new_width = int(img.height * target_ratio)
        left = (img.width - new_width) // 2
        img = img.crop((left, 0, left + new_width, img.height))
    else:
        new_height = int(img.width / target_ratio)
        top = (img.height - new_height) // 2
        img = img.crop((0, top, img.width, top + new_height))

    return img.resize((target_w, target_h), Image.LANCZOS)

# ---------------------------
# 📄 Process PDFs
# ---------------------------
for file in os.listdir(input_folder):
    if file.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, file)
        doc = fitz.open(pdf_path)

        if len(doc) == 0:
            continue

        # Render first page
        page = doc[0]
        pix = page.get_pixmap(dpi=200)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        base_name = Path(file).stem

        # Save each size into its own folder, with dimensions in filename
        for size, folder in sizes.items():
            resized_img = crop_center_resize(img, size)
            w, h = size
            out_path = os.path.join(base_output, folder, f"{base_name}_{w}x{h}.jpg")
            resized_img.save(out_path, "JPEG", quality=90)

print("✅ PDFs converted & saved into separate folders with size suffix in filename!")


✨ Welcome to the PDF to Resized Images Converter! ✨
Let's turn your PDFs into beautifully resized images.



📂 Enter the path to the folder containing PDFs:  Z:\4_Metadata\Uganda Airlines - UR\2025\1025\RAVE\Magazines
📂 Enter the path where resized images should be saved:  Z:\4_Metadata\Uganda Airlines - UR\2025\1025\RAVE\Magazines



🎨 Now, let's define your target image sizes.


➡️ Enter size (WIDTHxHEIGHT) or 0 to finish:  600x800


✅ Added size: 600x800



➡️ Enter size (WIDTHxHEIGHT) or 0 to finish:  0


✅ PDFs converted & saved into separate folders with size suffix in filename!
